In [ ]:
# Import dependencies

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import numpy as np
import math
import matplotlib.pyplot as plt
from plotly.offline import iplot
import pandas as pd
from scipy.interpolate import griddata

# define and plot the wing model with pressure map
# task = ['CP_nofilter', 'CP_filter']
task = 'CP_nofilter'

In [ ]:
def plotWing(xx,yy,vg,gridsz,pset):
    # pset is to set the angle of the model in degree
    chord = 1
    grsize = gridsz 

    c_left  = np.linspace(0,1,int(grsize/2)).reshape(int(grsize/2),1)
    c_right = np.linspace(1,0,int(grsize/2)).reshape(int(grsize/2),1)
    c  = np.concatenate((c_left,c_right),axis=0)
    c  = c*chord
    zz = np.zeros((len(xx),len(xx)))

    # interpolation, to find the z values of naca0012, base on x and y
    for ii in range(0,grsize):
        zz[0:sum(1*(yy[:,ii]<=c[ii])),ii]=np.flipud(5*0.12*c[ii]*(
            0.2969*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(1/2)
            -0.1260*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(1)
            -0.3516*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(2)
            +0.2843*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(3)
            -0.1015*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(4)))

    # remove excess
    zz[0,0] = 0

    for i in range(0,grsize):
        j = 0
        while zz[j,i] != 0:
            j = j+1
        if i == int(grsize/2) + 2 or i == int(grsize/2) + 1:
            xx[j+4:-1,i] = math.nan
            yy[j+4:-1,i] = math.nan
            zz[j+4:-1,i] = math.nan
        elif i == int(grsize/2) - 2 or i == int(grsize/2) - 1:
            xx[j+4:-1,i] = math.nan
            yy[j+4:-1,i] = math.nan
            zz[j+4:-1,i] = math.nan
        else:
            xx[j+3:-1,i] = math.nan
            yy[j+3:-1,i] = math.nan
            zz[j+3:-1,i] = math.nan

    lx = np.linspace(-0.3,0,grsize)
    ly = np.linspace(0,0.3,grsize)
    lxN = np.linspace(0,0.3,grsize)
    lyN = np.linspace(0.3,0,grsize)

    zzb = -zz
    zz  = zz  - 0.07*chord
    zzb = zzb - 0.07*chord

    # rotate translate
    yoffset = chord/2
    yy = yy - yoffset

    yy  = yy*math.cos(math.radians(pset)) - zz*math.sin(math.radians(pset))
    zz  = yy*math.sin(math.radians(pset)) + zz*math.cos(math.radians(pset))
    zzb = yy*math.sin(math.radians(pset)) + zzb*math.cos(math.radians(pset))

    yy = yy + yoffset
    yy = yy + 1.2*chord
    xx = xx + 0.02

    plotly.offline.init_notebook_mode()

    tmp = np.flipud(vg)
    vg  = np.fliplr(tmp)

    zz_up = np.nan_to_num(zz, nan=0)
    
    wing_model_up  = go.Surface(x=xx,y=yy,z=zz,showscale=False, surfacecolor=zz_up, colorscale='gray', opacity=1,
                               lighting=dict(ambient=0.5, diffuse=0, specular=0.5)
                               )      # upper section of the wing model

    # extract pressure map from zz --> upper surface
    zz_map = zz
    # Define the index of the three vertices of the triangle: the ports area (check the vg to find these indexes)
    x1, y1 = (int(grsize*0.2)+1, int(grsize*0.15))
    x2, y2 = (int(grsize*0.2)+1, int(grsize*0.45)-1)
    x3, y3 = (int(grsize*0.8)-1, int(grsize*0.45)-1) 
    # Create a boolean mask with the same shape as arr
    mask = np.zeros_like(zz_map, dtype=bool)
    # Iterate over the array and set all elements inside the triangle to True and outside to False
    for i in range(zz_map.shape[0]): # row
        for j in range(zz_map.shape[1]): # column
            if (i > min(x1,x2,x3) and i < max(x1,x2,x3)) and (j > min(y1,y2,y3) and j < max(y1,y2,y3)) and( i < (int(grsize*0.2)+1)+(((int(grsize*0.8)-1)-(int(grsize*0.2)+1))/((int(grsize*0.45)-1)-int(grsize*0.15)))*(j-int(grsize*0.15))):
                mask[i,j] = True
    zz_map[~mask] = np.nan

    zz_down = np.nan_to_num(zzb, nan=0)
    
    # down section of the wing model
    wing_model_down  = go.Surface(x=xx,y=yy,z=zzb,showscale=False, surfacecolor=zz_down, colorscale='gray', opacity=1,
                                 lighting=dict(ambient=0.9))     
    colorbar=dict(ticks='outside', ticklen=3, tickwidth=3, tickcolor='black', tickfont=dict(size=15,color='black'),
                    thickness=20, tickformat='.2f',
                    len=0.6,
                    xanchor='left', x=0.99, yanchor='top', y=0.8)

    # because there is latex equation in the text, the font size does not work, instead, use \large, \huge to control the latex size
    annotations=[ # I don't know why latex doesn't work; Here use heml symbol instead
                dict(x=0.1, y=0.9, xref="paper", yref="paper", text='<i>t<sup>*</sub>=2.8</i>',
                     font=dict(size=30, color='black'),showarrow=False),
                dict(x=1.042, y=0.83, xref="paper", yref="paper", text="<i>C<sub>P</sub></i>",
                     font=dict(size=20,color='black'),showarrow=False)
                ]
    #pressure map on the wing model
    wing_model_p_map = go.Surface(x=xx,y=yy,z=zz_map+0.01,surfacecolor=vg,colorscale='Jet',showscale=True,
                                 lighting=dict(ambient=0.9), colorbar = colorbar) 

    # plot the pressure ports on the wing wurface
    x_port = [-0.086, # 0.0957=(0.66-0.086) / 6 = 0.0957      (0.676-0.086)=0.0983
         -0.086-0.0957*0, -0.086-0.0957*1, -0.086-0.0957*2,
         -0.086-0.0957*0, -0.086-0.0957*1, -0.086-0.0957*2, -0.086-0.0957*3, -0.086-0.0957*4,
         -0.086-0.0983*0,                  -0.086-0.0983*2, -0.086-0.0983*3, -0.086-0.0983*4, -0.086-0.0983*5, -0.086-0.0983*6]
    y_port = [2-0.2*0, # (2-1.4)/3=0.2
         2-0.2*1, 2-0.2*1, 2-0.2*1,
         2-0.2*2, 2-0.2*2, 2-0.2*2, 2-0.2*2, 2-0.2*2,
         2-0.2*3,          2-0.2*3, 2-0.2*3, 2-0.2*3, 2-0.2*3, 2-0.2*3]
    z_port_calib = 0.008
    z_port = [-0.01,
             -0.007+z_port_calib, -0.012+z_port_calib, -0.022+z_port_calib,
             -0.016+z_port_calib, -0.018+z_port_calib, -0.02+z_port_calib, -0.024+z_port_calib, -0.030+z_port_calib,
             -0.033,        -0.034, -0.035, -0.036, -0.038, -0.041]
    points = {}
    for i in range(len(x_port)):
        points[i] = [x_port[i],y_port[i],z_port[i]]
    points = list(points.values())

    # Add the interpolated points to the plot
    rect_colors = ['red', 'green', 'blue', 'purple', 'orange']
    # Create an empty list to store the interpolated points and rectangles trace
    scatter_points_rect = []
    for i, point in enumerate(points):
        scatter_points_rect.append(go.Scatter3d(x=[point[0]], y=[point[1]], z=[point[2]], 
                                                  mode='markers',marker=dict(size=5, color='black', symbol='circle',
                                                  line=dict(color='black', width=1)), showlegend=False))
    # plot the wing
    fig = go.Figure(data=[wing_model_up, wing_model_down, wing_model_p_map]+scatter_points_rect)

    zoom_in_ration = 0.28  
    fig.update_layout(scene = {"xaxis": {"visible":False},
                               "yaxis": {"visible":False},
                               "zaxis": {"visible":False},
                               'camera_eye': {"x": -1*zoom_in_ration, "y": -2*zoom_in_ration, "z": 2*zoom_in_ration}, # best (-1 -2 2)
                               "aspectratio": {"x": 1, "y": 0.5, "z": 0.06}
                              },
                      scene_camera = {"center": {"x":-0.18, "y":0, "z":-0.1}},
                      width=800, height=800,
                      annotations=annotations)
    iplot(fig)
    # fig.show()
    # save figure as svg
    if task == 'CP_nofilter':
        fig.write_image("deltawing_noFilter.svg", scale=2)
    elif task == 'CP_filter':
        fig.write_image("deltawing_beselFilter.svg", scale=2)
    else:
        print('Error. Please define which task to run.')

print('plotWing funciton is done!')

In [ ]:
#####################################################
# Suppress/hide the warning
np.seterr(invalid='ignore')

CP_origin = pd.read_csv('CP_plot_map.csv')
CP_origin.set_index('case_number', inplace=True)
if task == 'CP_nofilter':
    CP_show = CP_origin.loc[:, CP_origin.columns.str.startswith('CP_nofilter')]
elif task == 'CP_filter':
    CP_show = CP_origin.loc[:, CP_origin.columns.str.startswith('CP_filter')]
else:
    print('Error. Please define which task to run.')

CP_show = CP_show.to_numpy()
L = 2500
caseToPlot = 13
# please add two texts in the figure with different positions
# input variables       can you give me an example using  direction      how to control the light        plot 3d surface using plotly
U = 1
Uf = 1.5
chord_real = 0.3
T = 0.3 # s periodic of the gust = 0.3s
sampleRate = 1000
T_s = 1/sampleRate
t = np.linspace(0,T_s*L,L)
t_star = t/T

# there are the positions of the 15 pressure ports, used for interpolation
# no p11, which is linked to stagnation point
x = [ 0.1, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.4, 0.5, 0.1, 0.3, 0.4, 0.5, 0.6, 0.7]
y = [ 0.2, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]

# plot the 3D model and the pressure contour
# port 16 links to stagnation point, so no use port 16
data_P_temp = CP_show[(L*(caseToPlot-1)):(L*caseToPlot),0:15]

i = 840 # only plot 550th point, for steady state
p1 = data_P_temp[i-1,:]
chord = 1
grSize = 1000  # 500 or 1000 are better
[xx,yy]=np.meshgrid(np.linspace(-1,1,grSize),np.linspace(0,1,grSize)) # mesh grid, a rectangle
points = np.concatenate((np.array(x).reshape(15,1), np.array(y).reshape(15,1)), axis=1)
vg = griddata(points,p1,(xx,yy),method='cubic')

plotWing(xx,yy,vg,grSize,0)

print('Done!')